In [1]:
import pytesseract
import cv2
import PyPDF2
import re
import os
import pandas as pd
import numpy as np
from pdf2image import convert_from_path

In [2]:
# Get all files recursively
top_dir = "/Users/timvigers/Desktop/G4 from Epic Rotated/Craig_Jessica"
files_only = []
for path, subdirs, files in os.walk(top_dir):
    for name in files:
        if ".pdf" in name:
            files_only.append(str(os.path.join(path, name)))

In [75]:
# Define function for image reading with and without thresholding
def get_text(img,threshold = True):
    if threshold == True:
        gry = cv2.cvtColor(np.array(img), cv2.COLOR_BGR2GRAY)
        flt = cv2.adaptiveThreshold(gry,300, cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY, 15, 16)
        text = pytesseract.image_to_string(flt)
    else:
        text = pytesseract.image_to_string(img)
    text = re.sub("[=!?',;.\n|+ ]",'',text)
    return(text)
# Function for searching both texts with regex
def regex(text,text2,expression):
    var = re.search(expression, text)
    if type(var) != re.Match:# Check missing and re-try without image thresholding
        var = re.search(expression, text2)    
    var = re.findall('\d+', var[0])   
    return(var)

In [76]:
results={'file': [], 'avg_glucose': [], 'sens_use': [], 'calib': [], 'sd': [],'high': [], 'target': [], 'low': []}
for file in files_only:
    results['file'].append(re.sub('/Users/timvigers/Desktop/G4 from Epic Rotated/','',file))
    # List PDF pages
    pages = convert_from_path(file)
    # Page 1
    t1 = get_text(pages[0])
    t2 = get_text(pages[0],threshold=False)
    # Pull all variables
    # Glucose
    avg_glucose = regex(t1,t2,'AverageGlucose\d*')   
    results['avg_glucose'].append(''.join(avg_glucose))
    # Sensor use
    sens_use = regex(t1,t2,'\d{1,2}of14Days')
    results['sens_use'].append(' of '.join(sens_use))
    # Calibrations
    calib = regex(t1,t2,'Calibrations/day\d*')
    results['calib'].append(''.join(calib))
    # SD
    sd = regex(t1,t2,'StandardDeviation\d*')   
    results['sd'].append(''.join(sd))
    # Percent high
    high = regex(t1,t2,'\d*%High')
    results['high'].append(''.join(high))
    # Percent target
    target = regex(t1,t2,'\d*%Target')
    results['target'].append(''.join(target))
    # Percent low
    low = regex(t1,t2,'\d*%Low')
    results['low'].append(''.join(low))
results

{'file': ['Craig_Jessica/Craig_4.3.14- 4.16.14.pdf',
  'Craig_Jessica/Craig_5.29.14- 6.11.14.pdf',
  'Craig_Jessica/Craig_8.22.14- 9.4.14.pdf',
  'Craig_Jessica/Craig_7.25.14- 8.7.14.pdf',
  'Craig_Jessica/Craig_9.18.14- 10.1.14.pdf',
  'Craig_Jessica/Craig_4.29.14- 5.12.14.pdf',
  'Craig_Jessica/Craig_6.13.14- 6.26.14.pdf',
  'Craig_Jessica/Craig_8.8.14- 8.21.14.pdf',
  'Craig_Jessica/Craig_8.23.14- 9.5.14.pdf',
  'Craig_Jessica/Craig_6.26.14- 7.9.14.pdf',
  'Craig_Jessica/Craig_7.1.14- 7.23.14.pdf'],
 'avg_glucose': ['136',
  '120',
  '142',
  '152',
  '133',
  '120',
  '119',
  '128',
  '140',
  '126',
  '123'],
 'sens_use': ['10 of 14',
  '14 of 14',
  '12 of 14',
  '44 of 14',
  '9 of 14',
  '12 of 14',
  '14 of 14',
  '13 of 14',
  '12 of 14',
  '12 of 14',
  '6 of 14'],
 'calib': ['', '', '12', '', '', '18', '15', '13', '13', '18', '20'],
 'sd': ['', '82', '53', '55', '49', '52', '43', '75', '52', '46', ''],
 'high': ['19', '10', '18', '28', '18', '13', '10', '16', '17', '13', '